In [1]:
using FastAI
using FluxTraining

┌ Info: Precompiling FastAI [5d0beca9-ade8-49ae-ad0b-a3cf890e669f]
└ @ Base loading.jl:1313


In [13]:
dataset = Datasets.loaddataset("imagenette2-160");

method = ImageClassification(Datasets.loadclasses("imagenette2-160"), (160, 160))

dls = methoddataloaders(dataset, method)

model = methodmodel(method, Models.xresnet18());

learner = Learner(model, dls, ADAM(), methodlossfn(method), ToGPU(), Metrics(SmoothLoss(), accuracy))

Learner()

In [14]:
fitonecycle!(learner, 2)

Epoch 1 TrainingPhase(): 100%|██████████████████████████| Time: 0:00:34


┌───────────────┬───────┬─────────┬────────────┬──────────┐
│         Phase │ Epoch │    Loss │ SmoothLoss │ Accuracy │
├───────────────┼───────┼─────────┼────────────┼──────────┤
│ TrainingPhase │   1.0 │ 1.68799 │    1.98817 │  0.43146 │
└───────────────┴───────┴─────────┴────────────┴──────────┘


Epoch 1 ValidationPhase(): 100%|████████████████████████| Time: 0:00:03


┌─────────────────┬───────┬─────────┬────────────┬──────────┐
│           Phase │ Epoch │    Loss │ SmoothLoss │ Accuracy │
├─────────────────┼───────┼─────────┼────────────┼──────────┤
│ ValidationPhase │   1.0 │ 1.65383 │    1.32354 │  0.50542 │
└─────────────────┴───────┴─────────┴────────────┴──────────┘


Epoch 2 TrainingPhase(): 100%|██████████████████████████| Time: 0:00:35


┌───────────────┬───────┬─────────┬────────────┬──────────┐
│         Phase │ Epoch │    Loss │ SmoothLoss │ Accuracy │
├───────────────┼───────┼─────────┼────────────┼──────────┤
│ TrainingPhase │   2.0 │ 1.10621 │    0.63056 │  0.64326 │
└───────────────┴───────┴─────────┴────────────┴──────────┘


Epoch 2 ValidationPhase(): 100%|████████████████████████| Time: 0:00:02


┌─────────────────┬───────┬─────────┬────────────┬──────────┐
│           Phase │ Epoch │    Loss │ SmoothLoss │ Accuracy │
├─────────────────┼───────┼─────────┼────────────┼──────────┤
│ ValidationPhase │   2.0 │ 0.88542 │    0.83544 │  0.72093 │
└─────────────────┴───────┴─────────┴────────────┴──────────┘


Learner()

In [17]:
finetune!(learner, 1)

Epoch 1 TrainingPhase(): 100%|██████████████████████████| Time: 0:00:35


┌───────────────┬───────┬─────────┬────────────┬──────────┐
│         Phase │ Epoch │    Loss │ SmoothLoss │ Accuracy │
├───────────────┼───────┼─────────┼────────────┼──────────┤
│ TrainingPhase │   1.0 │ 0.91977 │    0.90744 │  0.70086 │
└───────────────┴───────┴─────────┴────────────┴──────────┘


Epoch 1 ValidationPhase(): 100%|████████████████████████| Time: 0:00:02


┌─────────────────┬───────┬─────────┬────────────┬──────────┐
│           Phase │ Epoch │    Loss │ SmoothLoss │ Accuracy │
├─────────────────┼───────┼─────────┼────────────┼──────────┤
│ ValidationPhase │   1.0 │ 0.86985 │    0.82125 │  0.71832 │
└─────────────────┴───────┴─────────┴────────────┴──────────┘


Epoch 1 TrainingPhase(): 100%|██████████████████████████| Time: 0:00:35


┌───────────────┬───────┬─────────┬────────────┬──────────┐
│         Phase │ Epoch │    Loss │ SmoothLoss │ Accuracy │
├───────────────┼───────┼─────────┼────────────┼──────────┤
│ TrainingPhase │   1.0 │ 0.90201 │    1.02704 │  0.70781 │
└───────────────┴───────┴─────────┴────────────┴──────────┘


Epoch 1 ValidationPhase(): 100%|████████████████████████| Time: 0:00:02


┌─────────────────┬───────┬─────────┬────────────┬──────────┐
│           Phase │ Epoch │    Loss │ SmoothLoss │ Accuracy │
├─────────────────┼───────┼─────────┼────────────┼──────────┤
│ ValidationPhase │   1.0 │ 0.83348 │    0.59924 │  0.73172 │
└─────────────────┴───────┴─────────┴────────────┴──────────┘


Learner()

## `finetune!`

In [7]:
"""
    finetune!(learner, nepochs[, base_lr = 0.002; kwargs...])

Behaves as [`fastai.Learner.fine_tune`](https://github.com/fastai/fastai/blob/master/fastai/callback/schedule.py#L151)
"""
function finetune!(
        learner,
        nepochs,
        base_lr = 0.002;
        freezeepochs = 1,
        trainlayers = [2,],
        lr_mult = 100,
        div = 5,
        pct_start = 0.3,
        kwargs...)
    
    model = learner.model
    try
        # freeze backbone and train head
        FluxTraining.model!(learner, freeze(model, freezelayers))
        fitonecycle!(learner, freezeepochs, base_lr, pct_start=0.99; kwargs...)
        
        # unfreeze
        FluxTraining.model!(learner, model)
        base_lr /= 2
        # TODO: use discriminative learning rates
        fitonecycle!(
            learner, nepochs, base_lr;
            div = div, pct_start = pct_start, kwargs...)
        return learner
    catch e
        FluxTraining.model!(learner, model)
        rethrow(e)
    end
end

finetune!

In [10]:
"""
    freeze(model, indices)

Freeze all parameters in `model`, except those in `model[indices]`.
"""
freeze(model, indices::AbstractVector) = FrozenModel(model, model -> model[indices])

freeze

In [8]:
using Flux

In [17]:
struct FrozenModel
    model
    fn
end

Flux.params(model::FrozenModel) = params(model.fn(model.model))

## `lrfind!`

In [27]:
using Animations

In [ ]:
"""
    lrfind!(learner; start_lr = 1e-7, end_lr = 10, num_it = 100, stop_div = true)

Run the learning rate finder for `num_it` steps, exponentially increasing
the learning rate from `start_lr` to `end_lr`.

The model and callbacks are reset to their previous state afterwards.
"""
function lrfind!(learner; start_lr = 1e-7, end_lr = 10, num_it = 100, stop_div = true)
    origmodel = deepcopy(cpu(learner.model))
    origcallbacks = learner.callbacks
    try
        schedule = Schedule([0, num_it], [start_lr, end_lr], Animations.expout)
        callbacks = [Recorder(), Scheduler(LearningRate => schedule)]
        
    catch e
        rethrow(e)
    finally
        model!(learner, origmodel)
        learner.callbacks = origcallbacks
    end
end

In [ ]:
struct LRFinderResult
    losses
    steepest
    mindiv10
end

In [ ]:
struct LRFinderPhase <: FluxTraining.Phases.AbstractTrainingPhase
    
end

In [ ]:
FastAI.setschedules!()

In [ ]:
function fitepochphase!(
        learner::Learner,
        phase::LRFinderResult)
    
    dataiter = getdataiter(TrainingPhase(), learner)
    if dataiter === nothing
        throw(CancelEpochException("No data found for phase $(typeof(phase))"))
    end
    
    
    
    metric = SmoothLoss(phase.\beta)
    metricscb = Metrics(metrics)
    
    withcallbacks(learner, metricscb, Scheduler()) do
        setschedules!(learner, LearningRate => schedule)
        
        bestloss = Inf
        for (i, batch) in zip(1:phase.num_step, dataiter)
            fitbatch!(learner, batch, phase)
            loss = FluxTraining.stepvalue(metric)
            smoothloss = 
            push!(phase.result.losses, smoothloss)
        end
    end
    
    
    
    
    
    
    schedule = Schedule([0, phase.num_it], [phase.start_lr, phase.end_lr], Animations.expout)
    oldscheduler = FastAI.setschedules!(learner, LearningRate => schedule)
    oldmetrics = FluxTraining.replacecallback!(learner, Metrics())

    handle(EpochBegin(), learner, phase)

    for (i, batch) in zip(1:phase.num_step, dataiter)
        fitbatch!(learner, batch, phase)
    end

    handle(EpochEnd(), learner, phase)
end

In [ ]:
withcallbacks(learner, Metrics(); clearstate = [:metrics]) do
    withschedules(learner, LearningRate => schedule)
    
end

In [1]:
"""
    withcallbacks(f, learner, callbacks...)

Run `f` with `callbacks` on `learner`. Existing callbacks on `learner` of
the same type as in `callbacks` are swapped during the execution of `f`.
"""
function withcallbacks(f, learner, callbacks...)
    origcallbacks = [FluxTraining.replacecallback!(learner, cb) for cb in callbacks]
    try
        f()
    catch e
        rethrow(e)
    finally
        for (i, cb) in origcallbacks
            if isnothing(cb)
                FluxTraining.removecallback!(learner, callbacks[i])
            else
                FluxTraining.replacecallback!(learner, cb)
            end
        end
    end
end

withcallbacks